In [65]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import json
import csv
%matplotlib inline
from queue import Queue, PriorityQueue

In [66]:
# x = pd.read_csv('tdata.csv',sep='\t', lineterminator='\r')
# x

In [67]:
# df = pd.read_csv('tdata.csv', header=None, error_bad_lines=False)
# df

In [86]:
def show_weighted_graph(networkx_graph, node_size, font_size, fig_size):
  # Allocate the given fig_size in order to have space for each node
  plt.figure(num=None, figsize=fig_size, dpi=80)
  plt.axis('off')
  # Compute the position of each vertex in order to display it nicely
  nodes_position = nx.spring_layout(networkx_graph) 
  # You can change the different layouts depending on your graph
  # Extract the weights
  edges_weights  = nx.get_edge_attributes(networkx_graph,'Average Time Taken') 
  # Draw the nodes 
  nx.draw_networkx_nodes(networkx_graph, nodes_position, node_size=node_size,  
                         node_color = ["orange"]*networkx_graph.number_of_nodes())
  # Draw only the edges
  nx.draw_networkx_edges(networkx_graph, nodes_position, 
                          edgelist=list(networkx_graph.edges), width=2)
  # Add the weights
  nx.draw_networkx_edge_labels(networkx_graph, nodes_position, 
                               edge_labels = edges_weights, font_size=font_size)
  # Add the labels of the nodes
  nx.draw_networkx_labels(networkx_graph, nodes_position, font_size=font_size, 
                          font_family='sans-serif')
  plt.axis('off')
  plt.show()
    
def load_data_from_file(filename):
    df = pd.read_csv(filename,header = None).replace(' "', '', regex = True).replace('"','',regex = True)
    df.columns = ['Starting Station','Ending Station','Tubeline','Average Time Taken','Main Zone','SecondaryZone']
    
    # Making Input a correct Pandas DataFrame edge-list.
    edG = nx.from_pandas_edgelist(df,'Starting Station', 'Ending Station', edge_attr=True)
    
    return nx.Graph(edG)

In [69]:
tube_graph = load_data_from_file('tdata.csv')
# show_weighted_graph(tube_graph, 2000, 45, (100,100))

# 2.1 Implementing Depth First Search (DFS)

In [70]:
# Your function implementing DFS
def construct_path_from_root(node, root, graph):
    """the non-recursive way!"""
    
    path_from_root = [node['label']]
    while node['parent']:
        node = node['parent']
        path_from_root = [node['label']] + path_from_root
        print(path_from_root)
    cost = nx.path_weight(graph, path_from_root, 'Average Time Taken')
    return path_from_root, cost


def my_depth_first_graph_search(nxobject, initial, goal, compute_exploration_cost=False, reverse=False):
    """the no-oop way!"""
    
    frontier = [{'label':initial, 'parent':None}]  
    explored = {initial}
    number_of_explored_nodes = 1 

    while frontier:
        node = frontier.pop() # popping from the right of the list
        number_of_explored_nodes += 1
        if node['label']==goal:
            if compute_exploration_cost:
                print('number of explorations = {}'.format(number_of_explored_nodes))
            return node

        neighbours = reversed(list(nxobject.neighbors(node['label']))) if reverse else nxobject.neighbors(node['label'])
        for child_label in neighbours:

            child = {'label':child_label, 'parent':node}
            if child_label not in explored:
                frontier.append(child) # added to the right of the list, so it is a LIFO
                explored.add(child_label)      
    return None

In [71]:
# You can call your function like this:  Canada Water to Stratford
sol_DFS = my_depth_first_graph_search(tube_graph, 'Euston' , 'Stepney Green', True) # nodes = 32 - Tooting Bec to Covent Garden
construct_path_from_root(sol_DFS, 'Euston', tube_graph)

number of explorations = 177
['Mile End', 'Stepney Green']
['Bow Road', 'Mile End', 'Stepney Green']
['Bromley-by-Bow', 'Bow Road', 'Mile End', 'Stepney Green']
['West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End', 'Stepney Green']
['Canning Town', 'West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End', 'Stepney Green']
['North Greenwich', 'Canning Town', 'West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End', 'Stepney Green']
['Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End', 'Stepney Green']
['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End', 'Stepney Green']
['Bermondsey', 'Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End', 'Stepney Green']
['London Bridge', 'Bermondsey', 'Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End', 'Ste

(['Euston',
  'Warren Street',
  'Goodge Street',
  'Tottenham Court Road',
  'Leicester Square',
  'Charing Cross',
  'Embankment',
  'Westminster',
  'Green Park',
  'Victoria',
  'Pimlico',
  'Vauxhall',
  'Stockwell',
  'Oval',
  'Kennington',
  'Elephant & Castle',
  'Borough',
  'London Bridge',
  'Bermondsey',
  'Canada Water',
  'Canary Wharf',
  'North Greenwich',
  'Canning Town',
  'West Ham',
  'Bromley-by-Bow',
  'Bow Road',
  'Mile End',
  'Stepney Green'],
 56)

In [72]:
# # FINDING NEIGHBOUR NODES
# nei = tube_graph.neighbors('Canada Water')
# # nei = nx.neighbors(tube_graph, 'Amersham')
# for i in nei:
#     print([i])

# 2.1 Implementing Breath First Search (BFS)

In [73]:
# Your function implmenting BFS

def my_breadth_first_graph_search(nxobject, initial, goal, compute_exploration_cost=False, reverse=False):
    
    if initial == goal: # just in case, because now we are checking the children
        return None
    
    number_of_explored_nodes = 1    
    frontier = [{'label':initial, 'parent':None}]  
    # FIFO queue should NOT be implemented with a list, this is slow! better to use deque
    explored = {initial}
    
    while frontier:
        node = frontier.pop() # pop from the right of the list
        
        neighbours = reversed(list(nxobject.neighbors(node['label']))) if reverse else nxobject.neighbors(node['label'])

        for child_label in neighbours:
            child = {'label':child_label, 'parent':node}
            if child_label==goal:
                if compute_exploration_cost:
                    print('number of explorations = {}'.format(number_of_explored_nodes))
                return child
 
            if child_label not in explored:
                frontier = [child] + frontier # added to the left of the list, so a FIFO!
                number_of_explored_nodes += 1
                explored.add(child_label)
            
    return None

In [74]:
# call your function like this: 
sol_BFS = my_depth_first_graph_search(tube_graph, 'Canada Water' , 'Stratford', True) # nodes = 32 - Tooting Bec to Covent Garden
construct_path_from_root(sol_BFS, 'Stratford', tube_graph)

number of explorations = 247
['West Ham', 'Stratford']
['Canning Town', 'West Ham', 'Stratford']
['North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
['Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']
['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']


(['Canada Water',
  'Canary Wharf',
  'North Greenwich',
  'Canning Town',
  'West Ham',
  'Stratford'],
 15)

In [75]:
# You can call both your BFS and DFS function in this cell with large_labyrinth, "entrance" and "exit".

print('\nUsing BFS:\n'+'='*10)
print(construct_path_from_root(sol_DFS, 'Baker Street', tube_graph))


Using BFS:
['Mile End', 'Stepney Green']
['Bow Road', 'Mile End', 'Stepney Green']
['Bromley-by-Bow', 'Bow Road', 'Mile End', 'Stepney Green']
['West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End', 'Stepney Green']
['Canning Town', 'West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End', 'Stepney Green']
['North Greenwich', 'Canning Town', 'West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End', 'Stepney Green']
['Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End', 'Stepney Green']
['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End', 'Stepney Green']
['Bermondsey', 'Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End', 'Stepney Green']
['London Bridge', 'Bermondsey', 'Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Bromley-by-Bow', 'Bow Road', 'Mile End', 'Stepney Green']
['Bo

# 2.1 Implementing Uniform Cost Search (UCS)

In [76]:
import heapq

In [77]:
def uniformCostSearch(graph, origin, goal):
       frontier = []
       frontierIndex = {}
       node = (0, origin, [origin])
       # Use a dictionary to keep track of the elements inside the frontier (queue)
       frontierIndex[node[1]] = [node[0], node[2]]
       # Insert the node inside the forontier (queue)
       heapq.heappush(frontier, node)
       explored = set()
       while frontier:
              if len(frontier) == 0:
                     return None
              # Pop elemenet with lower path cost in the queue
              node = heapq.heappop(frontier)
              # Delete from the dicitonary the element that has beeen popped
              del frontierIndex[node[1]]
              # Check if the solution has been found
              if node[1] == goal:
                     return node
              explored.add(node[1])
              # Get a list of all the child nodes of node
              neighbours = list(graph.neighbors(node[1]))
              path = node[2]
              for child in neighbours:
                     path.append(child)
                     # create the child node that will be inserted in frontier
                     childNode = (node[0] + graph.get_edge_data(node[1], child)["Average Time Taken"], child, path)
#                      print("frontier = {}".format(childNode))
                     # Check the child node is not explored and not in frontier thorugh the dictionary
                     if child not in explored and child not in frontierIndex:
                            heapq.heappush(frontier, childNode)
                            frontierIndex[child] = [childNode[0], childNode[2]]
                     elif child in frontierIndex: 
                            # Checks if the child node has a lower path cost than the node already in frontier
                            if childNode[0] < frontierIndex[child][0]:
                                   nodeToRemove = (frontierIndex[child][0], child, frontierIndex[child][1])
                                   frontier.remove(nodeToRemove)
                                   heapq.heapify(frontier)
                                   del frontierIndex[child]

                                   heapq.heappush(frontier, childNode)
                                   frontierIndex[child] = [childNode[0], childNode[2]]
                     path = path[:-1]

              
#               print("Explored {}".format(len(explored)))

In [109]:
sol_UCS = uniformCostSearch(tube_graph, 'Baker Street', 'Wembley Park')
print("SOLUTION: {}".format(sol_UCS))


SOLUTION: (13, 'Wembley Park', ['Baker Street', 'Finchley Road', 'Wembley Park'])


In [110]:
# construct_path_from_root(sol_UCS, 'Canada Water')

In [111]:
# show_weighted_graph(tube_graph, 5000, 45, (100,100))

In [112]:
stations = load_data_from_file('tdata.csv')
# show_weighted_graph(stations, 2500, 10, (50, 50))

# 2.3 Computing Cost Function

In [113]:
def compute_path_cost_tubeline(graph, path):
  """
    Compute cost of a path
  """
  tubelines = {}
  cost1 = 0
  cost2 = 0
  for index_city in range(len(path) - 1):
    tubelines[index_city] = graph[path[index_city]][path[index_city + 1]]["Tubeline"]
    cost1 += graph[path[index_city]][path[index_city + 1]]["Average Time Taken"]
  for index_city in range(len(tubelines) - 1):
    cost2 += 1 if not tubelines[index_city] == tubelines[index_city+1] else 0
  return cost1, cost2

In [114]:
# Function Call

x1,x2 = compute_path_cost_tubeline(tube_graph, sol_UCS[2])
print('average cost for Average Time Taken - {}'.format(x1))
print('new cost for change of tubelines - {}'.format(x2))


average cost for Average Time Taken - 13
new cost for change of tubelines - 0


# 2.4 Heuristic search

# A* (A-STAR)- HEURISTIC 1

In [23]:
# Needed to hide warnings in the matplotlib sections
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import lines

from ipywidgets import interact
import ipywidgets as widgets
from IPython.display import display
import time
import math

In [120]:
import re
from queue import PriorityQueue

def heuristic(pos,node): # Calculates the admissible heuristic of a node
    
    node = str(pos[node])
#     print(node)
    node = node.replace('[','') 
    node = node.replace(']','')
    node = node.strip()
    node = re.sub(r' {2,}' , ' ', node)
    x, y = node.split(' ', maxsplit=2) # Split values by ,
    x = float(x)
    y = float(y)
    return abs(x-9) + abs(y-9) # Return calculation of admissible heuristic (manhattan distance)

def Astar(graph, origin, goal):
    admissible_heuristics = {} 
    pos = nx.spring_layout(graph) # positioning nodes using Fruchterman-Reingold force-directed algorithm
    h = heuristic(pos,origin)
    
    admissible_heuristics[origin] = h
    visited_nodes = {} # This will contain the data of how to get to any node
    visited_nodes[origin] = (h, [origin]) # I add the data for the origin node: "Travel cost + heuristic", "Path to get there" and "Admissible Heuristic"

    paths_to_explore = PriorityQueue()
    paths_to_explore.put((h, [origin], 0)) # Add the origin node to paths to explore, also add cost without h
        # I add the total cost, as well as the path to get there (they will be sorted automatically)

    while not paths_to_explore.empty(): # While there are still paths to explore
        # Pop elemenet with lower path cost in the queue
        _, path, total_cost = paths_to_explore.get()
        current_node = path[-1]
        neighbors = graph.neighbors(current_node) # I get all the neighbors of the current path
                
        for neighbor in neighbors:
            edge_data = graph.get_edge_data(path[-1], neighbor)
            if "Average Time Taken" in edge_data:
                cost_to_neighbor = edge_data["Average Time Taken"] # If the graph has weights
            else:
                cost_to_neighbor = 1 # If the graph does not have weights I use 1

            if neighbor in admissible_heuristics:
                h = admissible_heuristics[neighbor]
            else:
                h = heuristic(pos,neighbor)
                admissible_heuristics[neighbor] = h

            new_cost = total_cost + cost_to_neighbor
            new_cost_plus_h = new_cost + h
            if (neighbor not in visited_nodes) or (visited_nodes[neighbor][0]>new_cost_plus_h): # If this node was never explored, or the cost to get there is better than te previous ones
                next_node = (new_cost_plus_h, path+[neighbor], new_cost)
                visited_nodes[neighbor] = next_node # Update the node with best value
                paths_to_explore.put(next_node) # Also will add it as a possible path to explore

    return visited_nodes[goal] # return the goal information, it will have both the total cost and the path


In [121]:
sol_A_star = Astar(tube_graph,"Euston","Victoria")
print("Cost for Path - A-Star \n",sol_A_star)

Cost for Path - A-Star 
 (25.187218700000003, ['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria'], 7)


# NEW HEURISTIC (Main Zone and Manhattan Distance)

In [127]:
# Main zone Heuristic
def heuristic(G, a, b) -> float:
    if a == 0:
        return 0
    else:
        x = G[a][b]['Main Zone']
    return x

def heuristic1(nodes_position, node): # Calculates the admissible heuristic of a node
    # I know the format is [X,Y]
   
    node = str(nodes_position[node])
    node = node.replace('[','') # remove brackets
    node = node.replace(']','')
    node  = node.strip()
    node = re.sub(r' {2,}' , ' ', node)
    x, y = node.split(' ', maxsplit=2) # Split values by ,
    x = float(x)
    y = float(y)
    return abs(x-9) + abs(y-9) # Return calculation of admissible heuristic (manhattan distance)

In [130]:
def a_star_search(graph, start, goal):
    frontier = PriorityQueue()
    frontier.put(start, 0)
    h_list = {}
    h = heuristic(graph, 0, start)
    h_list[start] = h
    h1_list = {}
    nodes_position = nx.spring_layout(graph)
    h1 = heuristic1(nodes_position, start)
    h1_list[start] = h1
    came_from: Dict[str, Optional[str]] = {}
    cost_so_far: Dict[str, float] = {}
    came_from[start] = None
    path = [start]
    cost_so_far[start] = 0
   
    while not frontier.empty():
        current = frontier.get()
       
        if current == goal:
            break
       
        for next in graph.neighbors(current):
            h_list[next] = heuristic(graph, current, next) # calculating heuristic based on Mainzone changes in path
            h1_list[next] = heuristic1(nodes_position, next) # calculating heuristic based on manhattan distance
            new_cost = cost_so_far[current] +  graph[current][next]['Average Time Taken']
            if (next not in cost_so_far) or (new_cost < cost_so_far[next]):
                cost_so_far[next] = new_cost
                h = 0 if not h_list[current] == h_list[next] else 1 #add Mainzone heuristic value as 0 if there's a change in MainZone - lesser priority
                priority = new_cost + h + h1_list[next] #adding both heuristic to priority
                frontier.put(next, priority)
                came_from[next] = current
            if next not in path:
                path.append(next)
   
    return came_from, cost_so_far[goal]

In [132]:
start = 'Ealing Broadway'
goal = 'South Kensington'
came_from, cost_so_far = a_star_search(tube_graph, start , goal)

node = came_from[goal]
path = goal
while not node == None:
    path = node + ' ' + path
    node = came_from[node]
   
print(path, cost_so_far)

Ealing Broadway Ealing Common Acton Town Turnham Green Hammersmith Barons Court Earls' Court Gloucester Road South Kensington 20
